# Linear Models

- [Problem Definition](#problem-definition)
    - [Linear Model](#linear-model)
    - [Simplified CAPM](#simplified-capm)
- [Target Function](#target-function)
- [Raw Data](#raw-data)
- [Preprocessing](#preprocessing)
- [Machine Learning](#machine-learning)
    - [Training](#training)
    - [Model](#model)
    - [Error Function](#error-function)
- [Stochasic Gradient Descent](#stochastic-gradient-descent)
    - [`NumPy`](#sgd-numpy)
    - [`scikit-learn`](#sgd-sklearn)
    - [`TensorFlow`](#sgd-tf)
    - [`Keras`](#sgd-keras)
- [Maximum Likelihood Estimator](#maximum-likelihood-estimator)
    - [`NumPy`](#mle-numpy)
    - [`scikit-learn`](#mle-sklearn)
- [Challenge](#challenge)

## Problem Definition <a class="anchor" id="problem-definition"></a>

Provided a set $\mathcal{S}$ of $(x_{i}, y_{i})$ pairs: 
    $$\mathcal{S} = \{ (x_{1}, y_{1}), (x_{2}, y_{2}), \text{...}, (x_{k}, y_{k}) \}$$
Find a function $f$ that maps $x_{i} \rightarrow y_{i}$:
    $$f: \mathcal{X} \rightarrow \mathcal{Y}$$

$f$ is a:
* **(Discriminate) Model**
* **Map Function**
* **Function Approximation**    
for $\mathcal{S}$, such that:
    $$f(x_{i}) = \hat{y_{i}} \approx y_{i}, \quad \forall i$$

## Linear Model <a class="anchor" id="linear-model"></a>

$f$ is a **Linear Model**, iff:
\begin{equation}
    f(x_{i}) = w * x_{i}, \quad x_{i} \in \mathbb{R}^{n}; y_{i} \in \mathbb{R}^{m}; w \in \mathbb{R}^{m \times n}
\end{equation}

An example of the $k^{th}$ datapoint for $n=2$ and $m=1$ is:
$$ x_{k} = \begin{bmatrix}
               x_{k}^{(1)} & x_{k}^{(2)}
           \end{bmatrix}^{T}
         = \begin{bmatrix}
               1 & 2
           \end{bmatrix}^{T}
\text{,} \quad
   y_{k} = \begin{bmatrix}
               y_{k}^{(1)}
           \end{bmatrix}
         = \begin{bmatrix}
               3
           \end{bmatrix}
$$

For a finite $\mathcal{S}$ where $|\mathcal{S}| = k$, we express the above equation in matrix format as:
    $$f(\mathbf{X}) = \mathbf{\hat{y}} = \mathbf{X} * \mathbf{w},
    \quad \mathbf{X} \in \mathbb{R}^{k \times n}; \mathbf{y} \in \mathbb{R}^{k \times m}; \mathbf{w} \in \mathbb{R}^{n \times m}$$
    
Matrices $\mathbf{X}$ and $\mathbf{y}$ are constructed by stacking the $k$ pairs as row vectors.

### Simplified CAPM (Capital Asset Pricing Model ) <a class="anchor" id="simplified-capm"></a>

Factor models are a way of explaining the returns of one asset via a linear combination of the returns of other assets.
The general form of a factor model is:
\begin{equation}
    y = \alpha + \beta_{1} x_{1} + \beta_{2} x_{2} + \text{...} + \beta_{n} x_{n}
    \label{eq::beta}
\end{equation}
    
An asset's $\beta$ to another is just the $\beta$ factor from the equation above.

Look for:
    $$y_{stock} = \beta x_{benchmark} + \alpha = 
        \begin{bmatrix}
           x_{benchmark} & 1
       \end{bmatrix}
       \begin{bmatrix}
           \beta \\
           \alpha
       \end{bmatrix}$$

A sample of the dataset $S$ is visualized below:

| $x_{benchmark}$  | $y_{stock}$ |
| ---------------- | ----------- |
| 5.0              | 1.6         |
| 1.0              | 0.6         |
| 3.7              | 1.3         |
| 9.8              | 2.3         |
| 2.8              | 0.8         |

In [ ]:
# scientific computing library
import numpy as np
# machine learning library
import sklearn

# visualization tools
import matplotlib.pyplot as plt
import seaborn as sns

# show plots without need of calling `.show()`
%matplotlib inline

# prettify plots
plt.rcParams['figure.figsize'] = [20.0, 5.0]
sns.set_palette(sns.color_palette("muted"))
sns.set_style("ticks")

# supress warnings
import warnings
warnings.filterwarnings('ignore')

## Helper Functions

In [ ]:
def visualize(x, y=None, y_noise=None, y_hat=None):
    """Visualization helper function.
    
    Parameters
    ----------
    x: array-like
        x-axis values
    y: array-like
        y-axis target values
    y_noise: array-like
        y-axis observations
    y_hat: array-like
        y-axis model predictions
    """
    if y is not None:
        plt.plot(x, y, label='Target Function')
    if y_noise is not None:
        plt.plot(x, y_noise, 'o', label='Noisy Observations')
    if y_hat is not None:
        plt.plot(x, y_hat, '--o', label='Model Predictions')
    plt.legend();

## Target Function <a class="anchor" id="target-function"></a>

In [ ]:
N = 10
np.random.seed(0)

x = np.linspace(0, N, N)
y = 0.5 + 0.2 * x
visualize(x, y);

## Raw Data <a class="anchor" id="raw-data"></a>

In [ ]:
y_noise = y + np.random.normal(0,0.1, N)

visualize(x, y, y_noise)

## Preprocessing <a class="anchor" id="preprocessing"></a>

In [ ]:
X = np.ones((N, 2))
X[:, 1 ] = x

## Machine Learning <a class="anchor" id="machine-learning"></a>

Machine learning is the science of getting computers to act without being explicitly programmed.
Their course of action highly depends on the data that has been used in order to **train** them, while the generated program (a.k.a **model**) is a function of the data and the algorithm used to consume this data.

### Training <a class="anchor" id="training"></a>

Training is nothing more than the process of optimizating:
$$\mathcal{J}(w) = \min_{w}\bigg[ \mathbb{E} \big[ \mathcal{L}(\mathbf{y}, f(\mathbf{X} | w)) \big] + \lambda \mathcal{R}(w) \bigg]$$

where,
* $\mathcal{J}$: Objective Function
* $\mathcal{L}$: Loss Function
* $\mathcal{R}$: Regularization Term
* $f$: Mapping Function (`model`)
* $w$: `model` Parameters
* $S = \{ (x_{i}, y_{i}): i \in \mathcal{D} \}$

Assumming that $\mathcal{J}$ has a (global) minimum, such that:

$\DeclareMathOperator*{\argmin}{arg\,min}$
$$w^{*} = \argmin_{w}\bigg[ \mathbb{E} \big[ \mathcal{L}(\mathbf{y}, f(\mathbf{X} | w)) \big] + \lambda \mathcal{R}(w) \bigg]$$

where,
* $w^{*}$: Objective Function (Global) Minimizer

### Model <a class="anchor" id="model"></a>

Having found $w^{*}$ the `model` $f$ can be deployed and predictions can be made according to:

$$\mathbf{\hat{y}} = f(\mathbf{X} | w^{*}) \approx \mathbf{y}$$

In the case of Linear Models, $f$ gets the form of a linear function, with $w^{*}$ coefficients:

$$\mathbf{\hat{y}} = f(\mathbf{X} | w^{*}) = \mathbf{X} * w^{*} \approx \mathbf{y}$$

### Error Function <a class="anchor" id="error-function"></a>

Use **Mean Squared Error (MSE)** as the metric function in order to evaluate how good/bad the model $f$ is, such that:
    $$MSE_{f} = \frac{1}{k} \sum_{i=1}^{k} (y_{i} - \hat{y}_{i})^{2} = \frac{1}{2k} \sum_{i=1}^{k} (y_{i} - w_{i} * x_{i})^{2}$$
In matrix form:
    $$MSE_{f} = \frac{1}{k} (\mathbf{y} - \mathbf{X} * \mathbf{w})^{T}(\mathbf{y} - \mathbf{X} * \mathbf{w})$$

We can address the optimization problem of $MSE_{f}$:
* computationally (Stochastic Gradient Descent)
* analytically (Maximum Likelihood Estimator)

## Stochastic Gradient Descent <a class="anchor" id="stochastic-gradient-descent"></a>

Stochastic Gradient Descent (SGD) is the simplest and most used optimizer.
It is based on making incremental updates, towards the correctsolution,
using the derivative of the error at each step.

$$w_{t+1} = w_{t} - \eta (y_{i} - \hat{y}) x_{i}$$

### `NumPy` <a class="anchor" id="sgd-numpy"></a>

In [ ]:
# hyperparameters
n_epochs = 50
eta = 0.01

# parameters initialization
w = np.random.random(X.shape[1])

# training error over time
loss = []

# model fitting
for _ in range(n_epochs):
    y_hat = np.dot(X, w)
    l = 0.5 * np.sum((y_noise - y_hat)**2)
    i = np.random.randint(len(y_noise))
    error = y_hat[i] - y_noise[i]
    w = w - eta * error * X[i]
    loss.append(l)

# plot results
visualize(x, y, y_noise, y_hat)

# plot training error
plt.figure()
plt.plot(loss)
plt.title('Learning Curve')
plt.legend(['Training Error']);

### `scikit-learn`  <a class="anchor" id="sgd-sklearn"></a>

In [ ]:
from sklearn.linear_model import SGDRegressor

# hyperparameters
n_epochs = 100
eta = 0.01

# model initialization
model = SGDRegressor(loss='squared_loss', penalty='none', max_iter=n_epochs, learning_rate='constant', eta0=eta)

# model fitting & predictions
model.fit(X, y_noise)
y_hat = model.predict(X).ravel()

# plot results
visualize(x, y, y_noise, y_hat)

## Maximum Likelihood Estimator  <a class="anchor" id="maximum-likelihood-estimator"></a>

Differentiate $MSE_{f}(w)$ and find its minimum. There is a global minimiser, since $MSE_{f}$ is convex (quadratic).

$$\mathbf{w_{MLE}} = (\mathbf{X}^{T} \mathbf{X})^{-1} * \mathbf{X}^{T} * \mathbf{y}$$

### `NumPy` <a class="anchor" id="mle-numpy"></a>

In [ ]:
w = np.dot(np.dot(np.linalg.inv(np.dot(X.T, X)), X.T), y_noise)
y_hat = np.dot(X, w)

visualize(x, y, y_noise, y_hat)

### `scikit-learn` <a class="anchor" id="mle-sklearn"></a>

In [ ]:
from sklearn.linear_model import LinearRegression

# model initialization
model = LinearRegression()

# model fitting & predictions
model.fit(X, y_noise)
y_hat = model.predict(X).ravel()

# plot results
visualize(x, y, y_noise, y_hat)

## Challenges <a class="anchor" id="challenge"></a>

### Beta Hedging <a class="anchor" id="beta-hedging"></a>

Repeat the above experiment using the implementation you prefer most, on real world market data.<br>
Using the data provided in `{proj_dir}/notebooks/data/`:
* **S&P500 (SPY)**: as the market index (benchmark)
* **Apple (AAPL)**: as stock A
* **Google (GOOG)** as stocks B<br>

come up with a <u>Beta Hedging Strategy</u> to minimize risk.<br><br>

Load the data to a numpy.ndarray using the commands below:

In [ ]:
# helper loader function
loader = lambda asset: np.loadtxt('./data/%s.txt' % asset, delimiter=',')

# raw data
spy = loader('SPY')
aapl = loader('AAPL')
goog = loader('GOOG')

# normalised data
spy_ = spy / spy[0] - 1
aapl_ = aapl / aapl[0] - 1
goog_ = goog / goog[0] - 1

# visualization
plt.plot(spy_, label='S&P500')
plt.plot(aapl_, label='Apple Inc.')
plt.plot(goog_, label='Alphabet Inc.')
plt.title('Normalised Prices')
plt.ylabel('Normalised Change')
plt.xlabel('Time Index')
plt.legend();

In [ ]:
# regression plots
sns.regplot(spy_, aapl_, label='APPL')
sns.regplot(spy_, goog_, label='GOOG')
plt.title('Beta Exposure to S&P500')
plt.xlabel('SPY Percentage Change')
plt.ylabel('Stock Price Percentage Change')
plt.legend();

In [ ]:
# helper percentage return
pct_change = lambda series: np.diff(series) / (series[:-1] + 1e-6) # WARNING: prone to division by zero

aapl_pct = pct_change(aapl)
goog_pct = pct_change(goog)
spy_pct = pct_change(spy)

sns.distplot(aapl_pct, label='AAPL')
sns.distplot(goog_pct, label='GOOG')
sns.distplot(spy_pct, label='SPY')
plt.legend();